In [1]:
import numpy as np
import pandas as pd

In [2]:

DATAPATH = "/home/miki/Desktop/Deployment/bacillus-anthracis/data/raw/train.csv"
df = pd.read_csv(DATAPATH)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    48 non-null     object
 1   case    48 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 896.0+ bytes


In [4]:
df

,date,case
0,2014-01-31,132
1,2014-02-28,175
2,2014-03-31,240
3,2014-04-30,314
4,2014-05-31,299
5,2014-06-30,156
6,2014-07-31,253
7,2014-08-31,119
8,2014-09-30,188
9,2014-10-31,267


In [5]:
def makeXy(ts, nb_timesteps):
    """
    Input: 
           ts: original time series
           nb_timesteps: number of time steps in the regressors
    Output: 
           X: 2-D array of regressors
           y: 1-D array of target 
    """
    X = []
    y = []
    for i in range(nb_timesteps, ts.shape[0]):
        X.append(list(ts.loc[i-nb_timesteps:i-1]))
        y.append(ts.loc[i])
    X, y = np.array(X), np.array(y)
    return X, y

In [6]:
# Xtrain, ytrain = makeXy(df['case'], 12)

In [7]:
X_train, y_train = makeXy(df['case'], 11)
print('Shape of train arrays:', X_train.shape, y_train.shape)

Shape of train arrays: (37, 11) (37,)


In [8]:
df_val = pd.read_csv("/home/miki/Desktop/Deployment/bacillus-anthracis/data/raw/test.csv")

In [9]:
df_val

,date,case
0,2018-01-31,118
1,2018-02-28,97
2,2018-03-31,61
3,2018-04-30,34
4,2018-05-31,262
5,2018-06-30,68
6,2018-07-31,85
7,2018-08-31,60
8,2018-09-30,11
9,2018-10-31,3


In [10]:
X_val, y_val = makeXy(df_val['case'], 11)
print('Shape of validation arrays:', X_val.shape, y_val.shape)

Shape of validation arrays: (1, 11) (1,)


In [11]:
X_train, X_val = X_train.reshape((X_train.shape[0], X_train.shape[1], 1)), X_val.reshape((X_val.shape[0], X_val.shape[1], 1))
print('Shape of arrays after reshaping:', X_train.shape, X_val.shape)

Shape of arrays after reshaping: (37, 11, 1) (1, 11, 1)


In [39]:
import tensorflow as tf
from keras.models import *
from keras.callbacks import *
from keras.optimizers import *
from kerashypetune import KerasGridSearch
from keras.layers import Dense, LSTM,Flatten, TimeDistributed, RepeatVector, Dropout
import random 

def set_seed(seed):
    
    tf.random.set_seed(seed)
    np.random.seed(seed)
    random.seed(seed);


In [ ]:
lstm_model=Sequential()
lstm_model.add(LSTM(units=50, return_sequences=True,input_shape=(x_train_data.shape[1],1)))
lstm_model.add(Dropout(0.2))
lstm_model.add(LSTM(units=50, return_sequences=True))
lstm_model.add(Dropout(0.2))
lstm_model.add(LSTM(units=50))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(units=1))

model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
lstm_model.compile(loss='mean_absolute_error',optimizer='adam')
lstm_model.fit(X_train,y_train,epochs=10,batch_size=32,verbose=2)

lstm_model.save(company+".h5")

In [35]:
def get_model(param):
    
    set_seed(33)
            
    model = Sequential()
    model.add(LSTM(100, activation='relu', input_shape=(n_steps_in, n_features)))
    model.add(RepeatVector(n_steps_out))
    model.add(LSTM(100, activation='relu', return_sequences=True))
    model.add(TimeDistributed(Dense(1)))
    model.compile(optimizer='adam', loss='mse')
    
    return model

In [36]:
param_grid = {
    'unit_1': [128,64], 
    'unit_2': [64,32],
    'lr': [1e-2,1e-3], 
    'epochs': 100, 
    'batch_size': 512
}

In [37]:
es = EarlyStopping(patience=10, verbose=1, min_delta=0.001, monitor='val_accuracy', mode='auto', restore_best_weights=True)

hypermodel = get_model
# to pass external arguments to get_model use lambda function
# ==> hypermodel = lambda x: get_model(param=x, ...)
# callable(hypermodel) ==> True

kgs = KerasGridSearch(hypermodel, param_grid, monitor='val_accuracy', greater_is_better=True, tuner_verbose=1)
kgs.search(X_train, y_train, validation_data=(X_val, y_val), callbacks=[es])


8 trials detected for ('unit_1', 'unit_2', 'lr', 'epochs', 'batch_size')

***** (1/8) *****
Search({'unit_1': 128, 'unit_2': 64, 'lr': 0.01, 'epochs': 100, 'batch_size': 512})


TypeError: in user code:

    File "/home/miki/Desktop/Deployment/bacillus-anthracis/ant/lib/python3.8/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/home/miki/Desktop/Deployment/bacillus-anthracis/ant/lib/python3.8/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/miki/Desktop/Deployment/bacillus-anthracis/ant/lib/python3.8/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/home/miki/Desktop/Deployment/bacillus-anthracis/ant/lib/python3.8/site-packages/keras/engine/training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "/home/miki/Desktop/Deployment/bacillus-anthracis/ant/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered

In [23]:
OUTPUTPATH = "/home/miki/Desktop/Deployment/bacillus-anthracis/model"
hypermodel = get_model
es = EarlyStopping(patience=10, verbose=0, min_delta=0.001, monitor='val_loss', mode='auto', restore_best_weights=True)
kgs = KerasGridSearch(hypermodel, param_grid, monitor='val_loss', greater_is_better=False, tuner_verbose=1)
# save_weights_at = os.path.join(OUTPUTPATH, 'best_model.{epoch:02d}-{val_loss:.4f}.hdf5')
# save_best = ModelCheckpoint(save_weights_at, monitor='val_loss', verbose=0)
kgs.search(X_train, y_train, validation_data=(X_val ,y_val), callbacks=[es])


8 trials detected for ('unit_1', 'unit_2', 'lr', 'epochs', 'batch_size')


TypeError: __init__() missing 1 required positional argument: 'units'

In [40]:
model = Sequential()
model.add((LSTM( units=128, input_shape=(X_train.shape[1], X_train.shape[2])
    )
  )
)
model.add(Dropout(rate=0.2))
model.add(Dense(units=1))
model.compile(loss='mean_squared_error', optimizer='adam')

In [41]:
history = model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=32,
    validation_split=0.1,
    shuffle=False
)

Epoch 1/30
2/2 [==============================] - 8s 908ms/step - loss: 24499.9219 - val_loss: 19832.6348
Epoch 2/30
2/2 [==============================] - 0s 44ms/step - loss: 24471.9961 - val_loss: 19772.6914
Epoch 3/30
2/2 [==============================] - 0s 47ms/step - loss: 24351.3027 - val_loss: 19713.9941
Epoch 4/30
2/2 [==============================] - 0s 46ms/step - loss: 24312.1016 - val_loss: 19653.2305
Epoch 5/30
2/2 [==============================] - 0s 62ms/step - loss: 24234.3223 - val_loss: 19588.9883
Epoch 6/30
2/2 [==============================] - 0s 40ms/step - loss: 24188.9160 - val_loss: 19521.5625
Epoch 7/30
2/2 [==============================] - 0s 80ms/step - loss: 24080.4766 - val_loss: 19450.6074
Epoch 8/30
2/2 [==============================] - 0s 86ms/step - loss: 24036.7305 - val_loss: 19375.1445
Epoch 9/30
2/2 [==============================] - 0s 57ms/step - loss: 23962.5840 - val_loss: 19297.6113
Epoch 10/30
2/2 [==============================] - 0s 